# Nomograms

This notebook is designed to compute and save the clinical data used for nomograms.

In [ ]:
import env_apps

import os
import pandas as pd
import numpy as np

from constants import *
from src.data.datasets import Mask, TableDataset
from src.data.processing.sampling import extract_masks


We define our clinical features.

In [ ]:
AGE = Feature(column="AGE")
CLINICAL_STAGE = Feature(column="CLINICAL_STAGE", transform=MappingEncoding({"T1-T2": 0, "T3a": 1}))
CLINICAL_STAGE_MSKCC = Feature(
    column="CLINICAL_STAGE_MSKCC_STYLE",
    transform=MappingEncoding(
        {"T1c": 0, "T2": 0.2, "T2a": 0.2, "T2b": 0.4, "T2c": 0.6, "T3": 0.8, "T3a": 0.8, "T3b": 1}
    )
)
GLEASON_GLOBAL = Feature(column="GLEASON_GLOBAL")
GLEASON_PRIMARY = Feature(column="GLEASON_PRIMARY")
GLEASON_SECONDARY = Feature(column="GLEASON_SECONDARY")
PSA = Feature(column="PSA")


We extract the masks.

In [ ]:
masks = extract_masks(os.path.join(MASKS_PATH, "masks.json"), k=5, l=5)

We create the function that will be used to save the outer splits imputed dataframes.

In [ ]:
def save_outer_splits_dataframes(
    path_to_df: str, 
    path_to_folder: str, 
    stage_feature: Feature,
    masks: dict,
    mapping: dict = None
):
    learning_df = pd.read_csv(path_to_df)

    table_dataset = TableDataset(
        df=learning_df,
        ids_col=ID,
        tasks=TABLE_TASKS,
        cont_features=[AGE, PSA],
        cat_features=[stage_feature, GLEASON_GLOBAL, GLEASON_PRIMARY, GLEASON_SECONDARY]
    )
    
    for k, v in masks.items():
        train_mask, valid_mask, test_mask, inner_masks = v[Mask.TRAIN], v[Mask.VALID], v[Mask.TEST], v[Mask.INNER]
        table_dataset.update_masks(train_mask=train_mask, valid_mask=valid_mask, test_mask=test_mask)
        dataframe = table_dataset.imputed_df.copy()
        
        if mapping:
            dataframe[stage_feature.column] = dataframe[stage_feature.column].map(mapping)

        named_masks = {"train": train_mask, "valid": valid_mask, "test": test_mask}
        dataframe = pd.concat(
            objs=[dataframe.iloc[mask].assign(SETS=name) for name, mask in named_masks.items()],
            ignore_index=True
        )

        dataframe.to_csv(os.path.join(path_to_folder, f"outer_split_{k}.csv"), index=False)


We create the function that will be used to save the final dataframe.

In [ ]:
def save_final_dataframe(
    path_to_learning_df: str, 
    path_to_holdout_df: str, 
    path_to_folder: str, 
    stage_feature: Feature,
    masks: dict,
    mapping: dict = None
):
    learning_df = pd.read_csv(path_to_learning_df)
    holdout_df = pd.read_csv(path_to_holdout_df)

    df = pd.concat([learning_df, holdout_df], ignore_index=True)

    table_dataset = TableDataset(
        df=df,
        ids_col=ID,
        tasks=TABLE_TASKS,
        cont_features=[AGE, PSA],
        cat_features=[stage_feature, GLEASON_GLOBAL, GLEASON_PRIMARY, GLEASON_SECONDARY]
    )
    
    train_mask = list(range(len(learning_df)))
    test_mask = list(range(len(learning_df), len(learning_df) + len(holdout_df)))
    
    table_dataset.update_masks(
        train_mask=train_mask,
        test_mask=test_mask
    )
    
    dataframe = table_dataset.imputed_df.copy()
    if mapping:
        dataframe[stage_feature.column] = dataframe[stage_feature.column].map(mapping)

    named_masks = {"train": train_mask, "test": test_mask}
    dataframe = pd.concat(
        objs=[dataframe.iloc[mask].assign(SETS=name) for name, mask in named_masks.items()],
        ignore_index=True
    )

    dataframe.to_csv(os.path.join(path_to_folder, f"final_set.csv"), index=False)


## CAPRA

In [ ]:
save_outer_splits_dataframes(
    path_to_df="local_data/learning_table.csv", 
    path_to_folder="local_data/nomograms/CAPRA/",
    stage_feature=CLINICAL_STAGE,
    masks=masks,
    mapping={0: "T1-T2", 1: "T3a"}
)

In [ ]:
save_final_dataframe(
    path_to_learning_df="local_data/learning_table.csv",
    path_to_holdout_df="local_data/holdout_table.csv",
    path_to_folder="local_data/nomograms/CAPRA/",
    stage_feature=CLINICAL_STAGE,
    masks=masks,
    mapping={0: "T1-T2", 1: "T3a"}
)

## MSKCC

In [ ]:
save_outer_splits_dataframes(
    path_to_df="local_data/mskcc_learning_table.csv", 
    path_to_folder="local_data/nomograms/MSKCC/",
    stage_feature=CLINICAL_STAGE_MSKCC,
    masks=masks,
    mapping={0: "T1c", 0.2: "T2a", 0.4: "T2b", 0.6: "T2c", 0.8: "T3a", 1: "T3b"}
)

In [ ]:
save_final_dataframe(
    path_to_learning_df="local_data/mskcc_learning_table.csv",
    path_to_holdout_df="local_data/mskcc_holdout_table.csv",
    path_to_folder="local_data/nomograms/MSKCC/",
    stage_feature=CLINICAL_STAGE_MSKCC,
    masks=masks,
    mapping={0: "T1c", 0.2: "T2a", 0.4: "T2b", 0.6: "T2c", 0.8: "T3a", 1: "T3b"}
)

## CUSTOM

In [ ]:
save_outer_splits_dataframes(
    path_to_df="local_data/learning_table.csv", 
    path_to_folder="local_data/nomograms/CUSTOM/",
    stage_feature=CLINICAL_STAGE,
    masks=masks
)

In [ ]:
save_final_dataframe(
    path_to_learning_df="local_data/learning_table.csv",
    path_to_holdout_df="local_data/holdout_table.csv",
    path_to_folder="local_data/nomograms/CUSTOM/",
    stage_feature=CLINICAL_STAGE,
    masks=masks
)